Notebook objectives:
- Fine-tune a HuggingFace model for text classification by following this documentation. Below are the specifications:
  - Make use of Google Colab’s free GPU to train a HuggingFace model
  - Follow the documentation from start to finish
  - Be able to answer questions about each piece of code during the interview
  - Demonstrate fine-tuning using the sample dataset provided.
  - Bonus points: Use another text classification dataset to perform fine-tuning



# Project 1: Text classification

## Project objective
- Fine-tune a HuggingFace model for text classification 

## Set up install Desccription:
- transformers: model used for text classification
- dataset: library to download datasets with deep integration with hugging face
-  using  a pre trained ```distilbert-base-uncased``` finetuned to the  ```hate-speech-data``` data

-[hate-speech-data](https://huggingface.co/datasets/hate_speech_offensive) 

Why hate speech detection?
- Social media and messaging platforms have given us the ability to connect and express ourselves freely. However, what happens when these platforms are used to spread negativity and hate? 
- That's where the Hater_classifier comes in. This tool is designed to identify hateful speec, future us to filter out the hate speech making it a powerful tool for creating a positive online environment.



downloaded the libraries needed

In [5]:
! pip install  transformers
! apt install git-lfs
! pip install datasets
! pip install torch
! pip install imbalanced-learn
! pip install optuna
! pip install ray[tune]
! pip install sklearn
! pip install pynvml 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.9/dist-packages/pip/_vendor/pkg_resources/__init__.py", lin

import all the libraries needed

In [6]:
# create hugging face account, and get the api key so you can upload the model in  the hugging face website
from huggingface_hub import notebook_login

import pandas as pd 

# torch to use the gpu
import torch
# import datasets
#library for datasets where you can download data with just one simple code
from datasets import load_dataset, load_metric, Dataset, DatasetDict
# from transformers import AutoTokenizer
import transformers
from transformers import AutoTokenizer,Trainer,AutoModelForSequenceClassification, TrainingArguments
from transformers.utils import send_example_telemetry
# import imblearn
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from sklearn.model_selection import train_test_split

# for gpu util
from pynvml import * 



define functions to track GPU utilization

In [7]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()


Get your hugging face api in https://huggingface.co/settings/tokens so you can write /upload the model in the huggingface website

In [8]:
# log in ito  hugging face
notebook_login()



Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


checking the transformer version so we can know if there are issues

In [9]:
# show the version on the tranformer
# since transform is > 4.11.0 there is no issue
print(transformers.__version__)

4.27.4


In [10]:
send_example_telemetry("text_classification_notebook", framework="pytorch")

### Model Definition
Pick a model from the [Model Hub](https://huggingface.co/models) with a  clasffification head 

Adjust the batch size as needed to not run out of memory


why choose distilbert-base-uncased for hate speech detection?
- The model is pretrained on a large corpus of English data in a self-supervised fashion.
- The model learns an inner representation of the English language that can then be used to extract features useful for downstream tasks
- distilbert-base-uncased: 110m Params compared other bert-large-uncased which is  3x more (340M)

In [11]:
task = "hate_speech_detection"
model_checkpoint = "distilbert-base-uncased"
batch_size = 32

# from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
# model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

# inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
# with torch.no_grad():
#     logits = model(**inputs).logits

# predicted_class_id = logits.argmax().item()
# model.config.id2label[predicted_class_id]

Pick random entries to get a sense of what the data looks like
1. turn the into dataset object into a dataframae so you can get samples from it 

In [ ]:
# pd.DataFrame(data["train"]).sample(10)

### Loading the data for hate speech

data description :
- hate_speech_count:number of CF users who judged the tweet to be hate speech.

- offensive_language_count :number of CF users who judged the tweet to be offensive.

- neither_count: number of CF users who judged the tweet to be neither offensive nor non-offensive.

- tweet: the tweet being annotated

- class = class label for majority of CF users. 
  - 0 - hate speech 
  - 1 - offensive language 
  - 2 - neither


In [18]:
#
dataset_name = "hate_speech_offensive"
metric = load_metric('accuracy','hate_speech_offensive')

hate_speech_ds  = load_dataset(dataset_name)
# hate_speech = pd.read_csv("hate_speech_labeled_data.csv")

Generating train split:   0%|          | 0/24783 [00:00<?, ? examples/s]

Dataset hate_speech_offensive downloaded and prepared to /root/.cache/huggingface/datasets/hate_speech_offensive/default/1.0.0/5f5dfc7b42b5c650fe30a8c49df90b7dbb9c7a4b3fe43ae2e66fabfea35113f5. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [115]:
metric

Metric(name: "accuracy", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions (`list` of `int`): Predicted labels.
    references (`list` of `int`): Ground truth labels.
    normalize (`boolean`): If set to False, returns the number of correctly classified samples. Otherwise, returns the fraction of correctly classified samples. Defaults to True.
    sample_weight (`list` of `float`): Sample weights Defaults to None.

Returns:
    accuracy (`float` or `int`): Accuracy score. Minimum possible value is 0. Maximum possible value is 1.0, or the number of examples input, if `normalize` is set to `True`.. A higher score means higher accuracy.

Examples:

    Example 1-A simple example
        >>> accuracy_metric = datasets.load_metric("accuracy")
        >>> results = accuracy_metric.compute(references=[0, 1, 2, 0, 1, 2], predictions=[0, 1, 1, 2, 1, 0])
        >>> print(results)
        {'accuracy': 0.5}

   

In [116]:
hate_speech_ds

DatasetDict({
    train: Dataset({
        features: ['count', 'hate_speech_count', 'offensive_language_count', 'neither_count', 'class', 'tweet'],
        num_rows: 24783
    })
})

In [19]:
hate_speech_df = pd.DataFrame(hate_speech_ds["train"])

In [20]:
# hate_speech_df

Showing some samples of each class
- Hate Speech: language that is used to expresses hatred towards a targeted group or is intended to be derogatory, to humiliate, or to insult the members of the group

- offensive language:contains foul words but not associating it towards a person or a group of people

- Neither: as long as it doesn't have badwords it's not a identified as hate speech or offensive language 


(0: 'hate-speech', 1: 'offensive-language' or 2: 'neither') 

source: https://arxiv.org/abs/1703.04009

In [21]:
#adding to expand the tweet to see the full text
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_colwidth', -1)

print("Hate speech")
display(hate_speech_df[hate_speech_df["class"]==0].sample(10)[["class","tweet"]])
print("-----------------------------------------------------------------")

print("Offensive language")
display(hate_speech_df[hate_speech_df["class"]==1].sample(10)[["class","tweet"]])
print("-----------------------------------------------------------------")

print("Neither")
print()
display(hate_speech_df[hate_speech_df["class"]==2].sample(10)[["class","tweet"]])
print("-----------------------------------------------------------------")

Hate speech


<ipython-input-21-73231f90a4b2>:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,class,tweet
23537,0,ah nigga dat cares about opinions and get involve in hoe shit either Gay broke or ain't getting enough pussy!!
3815,0,@Kevin_McAdams Because Ian's retarded
14566,0,"RT @C_Phil_13: Man... I was trying to eat &#128555; ""@kylegotjokes: ""Nigga you said it was bitches out here, I showed up for nothin!"" http://t.co/u&#8230;"
1694,0,&#8220;@alicia_688: You aint special niglet RT @Vice504fly: When I'm chilling wit Greg ppl walk up to me to ask me to take pics&#8221;hatin
21037,0,Some niggas save hoes
11653,0,"If you kill all cops and cracker babies, what next?????? Black states, white states, and Latino states? Isn't that moving backwards?"
2297,0,@05proFESSOR I bet he's a Starbucks queer
8609,0,Damn @JustDoMe_ &amp; his nigger music waking up the whole school in the morning .. &#128664;&#127926;&#128543;
13660,0,Only Americans would hire white trash with ties to international terrorism as police officers.
23975,0,"i find it funny when fags write ""looking for bf"" on a grindr ap. LOL thats a fucking ap dumbass."


-----------------------------------------------------------------
Offensive language


,class,tweet
2831,1,@ChaseManhatttan you become retarded
18266,1,RT @_CollegeHumor_: One loyal girlfriend is worth more than one hundred hoes.
11707,1,Ill take yo bitch and make her everybody bitch&#128558;&#128074;
16830,1,"RT @OxDictionary: I got bitches""\n\nDefinition: I'm with some female friends who don't know that I'm referring to them as bitches."
2110,1,"' This be me &#1041204;&#1041204;&#1041204; \nCuh went from "" Its time to settle down "" to "" Where the hoes at ?! "" &#1041204;&#1041204;&#1041204; . http://t.co/bi9rbvjTUn"
735,1,#Lingo so A1 niccas be Tryn talk like me
22963,1,Woman have staring problems just as much as men..Talking all this mumbo jumbo about men..I just give them retarded faces or ignore them..
6170,1,"@infidelpamelaLC #Allah can KISS MY ASS!\nAnd yes, MUHAMMED was a bitch! Just like OSAMA BINLADEN!"
15969,1,"RT @KayciMalynn: @__0utcast___ hey cunt, I miss you. &#128557;&#128557;"
19713,1,"RT @morganx3blowin: I fucked your bitch , &#128523;&#128527; how you feel?"


-----------------------------------------------------------------
Neither



,class,tweet
3858,2,@Kummar_King I volunteer to be guinea pig and help test it out &#128527;&#128587;
17147,2,"RT @RussOnPolitics: GOP nominee for #MAgov Charlie Baker's ""fisherman story"" falls apart. He deserves an Oscar for his play crying. http://&#8230;"
16586,2,RT @NFLTalkRT: RT if your team is trash
12865,2,Mitt Romney hates big bird. Fascist.
4128,2,@Miseducated411: Your mind is free and your soul. satan will present to u in his kingdom the things that mock u. Stay strong in Yahshua...
13743,2,Overripe eggplants are not good eats. #UNCLEtweet
5926,2,@emilysteers WASP EGGS
22205,2,"To be clear, I don't think there's any chance Krug leaves for Russia. But it's fun to mock the speculation."
16724,2,RT @NuteIla: when I die on flappy bird http://t.co/HKNUTfDVJZ
21551,2,The fact that the #Bruins drafted a Yankees fan just goes to show how weak the scouting staff is IMO. Didn't do their homework. #firechoreli


-----------------------------------------------------------------


Dataset observations:
1. Majority (77%) of the examples are  offensive language 
2. We rarely find hate speech having only 1430 examples (5% of the whole dataset)

Possible experiments:
- Rebalance the dataset via:
  - oversampling 
  - undersampling 


In [22]:
#check the distribution of the classes
hate_speech_df["class"].value_counts(normalize=True)


1    0.774321
2    0.167978
0    0.057701
Name: class, dtype: float64

check for nulls and drop them

In [23]:
hate_speech_df.isna().sum()
hate_speech_df = hate_speech_df.dropna()



## Preprocessing

### Tokenize

1. Defining the tokenizer (tokenizing is a method or process to convert the words into numbers such that the computer can understand it)

2. sample of the tokenizer




In [24]:
#create a toeknizer that fits the model in this case bert-distil-uncase
#use the vocab of the pretrained model
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

#input is the id in of the vocabulary 
tokenizer("it's me, hi, I'm the problem, it's me")

{'input_ids': [101, 2009, 1005, 1055, 2033, 1010, 7632, 1010, 1045, 1005, 1049, 1996, 3291, 1010, 2009, 1005, 1055, 2033, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

tokenize curse words or profanity to check what's the result

In [25]:
print(tokenizer("faggot"))
print(tokenizer("bitch"))
# print(tokenizer("bitch"))

{'input_ids': [101, 6904, 13871, 4140, 102], 'attention_mask': [1, 1, 1, 1, 1]}
{'input_ids': [101, 7743, 102], 'attention_mask': [1, 1, 1]}
{'input_ids': [101, 7743, 102], 'attention_mask': [1, 1, 1]}


possible sentence keys:

In [31]:
hate_speech_ds

DatasetDict({
    train: Dataset({
        features: ['count', 'hate_speech_count', 'offensive_language_count', 'neither_count', 'class', 'tweet'],
        num_rows: 24783
    })
})


<!-- After tokenzing,use pytorch to train the model to take advantage of colab free gpu:

- video source: https://youtu.be/Dh9CL8fyG80
- link source: https://youtu.be/Dh9CL8fyG80 -->


tokenize onse sample of the training set

In [32]:
#
# sentence1_key, sentence2_key = task_to_keys[task]
sentence1_key = "tweet"

#print the sentence
print(f"Sentence: {hate_speech_ds['train'][0][sentence1_key]}")

#print the tokenized sentence 
tokenizer(hate_speech_ds['train'][0][sentence1_key],truncation=True) 

Sentence: !!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...


{'input_ids': [101, 999, 999, 999, 19387, 1030, 9815, 19454, 21818, 2135, 1024, 2004, 1037, 2450, 2017, 5807, 1005, 1056, 17612, 2055, 9344, 2039, 2115, 2160, 1012, 1004, 23713, 1025, 2004, 1037, 2158, 2017, 2323, 2467, 2202, 1996, 11669, 2041, 1012, 1012, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}



The argument:
- `truncation=True`: This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model.

Why did I tokenize first?
-  because its better to tokenize first then oversample and undersample, and also need to train test plit
- so don't need to retokenizing the same tweets

In [33]:
#pre process function for existing data from dataset library
def preprocess_function(examples):
  return tokenizer(examples["tweet"], truncation=True)

# #preprocessing on hate_speech dataset
# def my_preprocess_function(examples):
#   return tokenizer(examples, truncation=True)

#encoded hate_speech for fine tuning
# endocded_hspeech = hate_speech["tweet"].map(my_preprocess_function)

encoded_ds = hate_speech_ds.map(preprocess_function)

#check what encoded ds looks like
encoded_ds

Map:   0%|          | 0/24783 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['count', 'hate_speech_count', 'offensive_language_count', 'neither_count', 'class', 'tweet', 'input_ids', 'attention_mask'],
        num_rows: 24783
    })
})

In [34]:
hate_speech_df = pd.DataFrame(encoded_ds["train"])

In [35]:
# hate_speech_df

### Undersampling and Oversampling

1. Defining the tokenizer (tokenizer is a method or process to convert the words into numbers such that the computer can understand it)
2. sample of the tokenizer




Perform oversampling using [Imbalanced-Learn Library](https://machinelearningmastery.com/random-oversampling-and-undersampling-for-imbalanced-classification/)

-  ```minority``` parameter means,  majority of 1,000 examples and the minority class had 100, this strategy would oversampling the minority class so that it has 1,000 examples.

- Need to oversample twice becuase we have 3 classes, after the first over sampling only 0.0 has increased  from 2050 to 30333 (same logic applies to oversampling)



In [36]:
# eval
X = hate_speech_df[["tweet",'input_ids', 'attention_mask']].values
y = hate_speech_df["class"].values

print("distrubution before over sampling")
print(Counter(y))
#d 1,000 examples and the minority class had 100, this strategy would oversampling the minority class so that it has 1,000 examples.
oversample = RandomOverSampler(random_state=0,sampling_strategy='minority')

X_over, y_over = oversample.fit_resample(X, y)
print("distrubution after over sampling")
print(Counter(y_over))

# need to oversample twice becuase we have 3 classes
# after the first over sampling only 0.0 has increased  from 2050 to 30333
X_over, y_over = oversample.fit_resample(X_over, y_over)
print("distrubution after 2nd over sampling")
print(Counter(y_over))


over_h_speech = pd.DataFrame(X_over, columns=["tweet",'input_ids', 'attention_mask'])
over_h_speech["label"] = y_over

distrubution before over sampling
Counter({1: 19190, 2: 4163, 0: 1430})
distrubution after over sampling
Counter({1: 19190, 0: 19190, 2: 4163})
distrubution after 2nd over sampling
Counter({2: 19190, 1: 19190, 0: 19190})


In [37]:
# over_h_speech

Perform undersampling using [Imbalanced-Learn Library](https://machinelearningmastery.com/random-oversampling-and-undersampling-for-imbalanced-classification/)


In [38]:
undersample = RandomUnderSampler(random_state=0,sampling_strategy='majority')

X_under, y_under = undersample.fit_resample(X, y)
print("distrubution after under sampling")
print(Counter(y_under))

X_under, y_under = undersample.fit_resample(X_under, y_under)
print("distrubution after 2nd under sampling")
print(Counter(y_under))

# under_h_speech = pd.DataFrame({"tweet":X_under.flatten(),"class":y_under})


under_h_speech = pd.DataFrame(X_over, columns=["tweet",'input_ids', 'attention_mask'])
under_h_speech["label"] = y_over



distrubution after under sampling
Counter({2: 4163, 0: 1430, 1: 1430})
distrubution after 2nd under sampling
Counter({0: 1430, 1: 1430, 2: 1430})


### Train test split on the following:
- hate_speech
- over_h_speech
- under_h_speech

In [39]:
hate_speech_df.columns = ['count', 'hate_speech_count', 'offensive_language_count',
       'neither_count', 'label', 'tweet', 'input_ids', 'attention_mask']

In [40]:

train, test = train_test_split(hate_speech_df, test_size=0.2)
over_train, over_test = train_test_split(over_h_speech, test_size=0.2)
under_train, under_test = train_test_split(under_h_speech, test_size=0.2)


                               

### Create a Dataset Dictionary
- So we can use it for the model

reference: The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set (with more keys for the mismatched validation and test set in the special case of `mnli`).

In [41]:
# Dataset.from_pandas(over_train)



train_ds = Dataset.from_pandas(train, split="train")
test_ds = Dataset.from_pandas(test, split="test")

o_train_ds = Dataset.from_pandas(over_train, split="o_train")
o_test_ds = Dataset.from_pandas(over_train, split="o_test")


u_train_ds = Dataset.from_pandas(under_train, split="u_train")
u_test_ds = Dataset.from_pandas(under_test, split="u_test")



In [42]:
train_ds

Dataset({
    features: ['count', 'hate_speech_count', 'offensive_language_count', 'neither_count', 'label', 'tweet', 'input_ids', 'attention_mask', '__index_level_0__'],
    num_rows: 19826
})

In [43]:

#processed dataset
dataset_proc = DatasetDict({"train":train_ds,
                             "test":test_ds, 
                             "o_train_ds":o_train_ds,
                             "o_test_ds":o_test_ds,
                             "u_train_ds":u_train_ds,
                             "u_test_ds":u_test_ds, })


In [44]:
dataset_proc

## Fine-tuning the model

1. load the model into cache
2. .tocuda() to use gpu [source](https://huggingface.co/docs/transformers/perf_train_gpu_one)



In [45]:

labels = 3
# model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=labels)
##comment this out if you just want cpu
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=labels).to("cuda")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'pre_classifi

defining training arguments

required attributes
-  folder name for model checkpoint storage

- evaluation_strategy (str or IntervalStrategy, optional, defaults to "no") — The evaluation strategy to adopt during training. Possible values are:
- per_device_train_batch_size (int, optional, defaults to 8) — The batch size per GPU/TPU core/CPU for training.
- we can push the model to the Hub regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook

In [87]:
model_name = model_checkpoint.split("/")[-1]
batch_size= 32
metric_name = "accuracy"
task = "hate_speech"

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
)

measurement of performance/ prediction 
1. `metric` we loaded earlier
2. added a for loop for get the prediction, to get the label/position of the predicton 

other docs: https://huggingface.co/docs/transformers/v4.27.2/en/internal/trainer_utils#transformers.EvalPrediction

In [95]:
import numpy as np
#fixed accuracy
#added a for loop for get the prediction
def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    max_pred = []
    for x in predictions:
      max_pred.append(np.array(x).argmax())
    # predictions = predictions[:, 0]
    print(max_pred)
    print(labels)
    return metric.compute(predictions=max_pred, references=labels)

In [96]:
#define the validation key of sst2
# validation_key  = "validation"
trainer = Trainer(model,
                  args,
                  train_dataset=dataset_proc["train"],
                  eval_dataset=dataset_proc["test"],
                  tokenizer=tokenizer,
                  compute_metrics=compute_metrics)



/content/distilbert-base-uncased-finetuned-hate_speech is already a clone of https://huggingface.co/Dc26/distilbert-base-uncased-finetuned-hate_speech. Make sure you pull the latest changes with `repo.git_pull()`.


In [97]:
result = trainer.train()
print_summary(result)
#why is the training and loss and accuracy all the same?

# print_summary(result)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.014800,0.776547,0.899939
2,0.014700,0.783275,0.901957
3,0.016100,0.804942,0.903772
4,0.011900,0.853567,0.902562
5,0.004600,0.853168,0.904579


[1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 0, 1, 2, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 1, 2, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 0, 1, 1, 2, 1, 1, 2, 1, 1, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 0, 2, 1, 2, 1, 1, 2, 2, 1, 1, 1, 2, 1, 

In [ ]:
 # result

In [98]:
trainer.evaluate()

[1, 1, 1, 1, 2, 1, 0, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 0, 1, 2, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 1, 2, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 0, 1, 1, 2, 1, 1, 2, 1, 1, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 0, 2, 1, 2, 1, 1, 2, 2, 1, 1, 1, 2, 1, 

{'eval_loss': 0.8531681299209595,
 'eval_accuracy': 0.9045793826911438,
 'eval_runtime': 11.6719,
 'eval_samples_per_second': 424.694,
 'eval_steps_per_second': 13.28,
 'epoch': 5.0}

In [99]:
trainer.push_to_hub()

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 1.00/255M [00:00<?, ?B/s]

Upload file runs/Mar31_14-43-24_c2b043a0ffff/events.out.tfevents.1680275215.c2b043a0ffff.787.14:   0%|        …

Upload file runs/Mar31_14-43-24_c2b043a0ffff/events.out.tfevents.1680274354.c2b043a0ffff.787.12:   0%|        …

To https://huggingface.co/Dc26/distilbert-base-uncased-finetuned-hate_speech
   a2e1f83..6b0b97b  main -> main

   a2e1f83..6b0b97b  main -> main

To https://huggingface.co/Dc26/distilbert-base-uncased-finetuned-hate_speech
   6b0b97b..64d7e66  main -> main

   6b0b97b..64d7e66  main -> main



'https://huggingface.co/Dc26/distilbert-base-uncased-finetuned-hate_speech/commit/6b0b97b9a05816860dd65c1f310924a5ee67db18'

In [100]:
f"{model_name}-finetuned-{task}"

'distilbert-base-uncased-finetuned-hate_speech'

In [101]:
# how to share the model you created

AutoModelForSequenceClassification.from_pretrained("Dc26/distilbert-base-uncased-finetuned-hate_speech")

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

## Hyperparameter search

install optuna and raytune cause it's used for hyper parameter tuning

we also defined a new model using the ```new_model```, so we create a new model every time

In [102]:
num_labels=3

def new_model():
    return AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

In [103]:
trainer = Trainer(
    model_init=new_model,
    args=args,
    train_dataset=dataset_proc["train"],
    eval_dataset=dataset_proc["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'pre_classifi

hyperparameter search  may take a long time to run on the full dataset so 
- it only runs on 1/10th of the dataset
- only train the full dataset on the best perfoming one

In [104]:
train_dataset = dataset_proc["train"].shard(index=1, num_shards=10) 

In [105]:
# returns Best Run object which mazimizes the accuracy/ desired metric
best_run = trainer.hyperparameter_search(n_trials=5, direction="maximize")

[I 2023-03-31 15:09:02,157] A new study created in memory with name: no-name-8a1cb9c5-6810-496f-b0bd-de13bf2d2241
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpo

Epoch,Training Loss,Validation Loss,Accuracy
1,0.458100,0.399368,0.870284
2,0.357900,0.342197,0.885616
3,0.318600,0.322436,0.892274
4,0.307100,0.315369,0.894291


[1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 2, 2, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 

[I 2023-03-31 15:20:32,119] Trial 0 finished with value: 0.8942909017550938 and parameters: {'learning_rate': 1.0245760923880995e-06, 'num_train_epochs': 4, 'seed': 26, 'per_device_train_batch_size': 16}. Best is trial 0 with value: 0.8942909017550938.
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model f

Epoch,Training Loss,Validation Loss,Accuracy
1,0.299000,0.320764,0.904378
2,0.274200,0.323939,0.905386
3,0.232600,0.324972,0.908412
4,0.221500,0.328461,0.910833
5,0.206300,0.335083,0.910833


...

Epoch,Training Loss,Validation Loss,Accuracy
1,0.303200,0.255145,0.910833
2,0.214600,0.269593,0.907202
3,0.173600,0.298805,0.904176
4,0.140500,0.350465,0.907605


[1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 0, 1, 1, 1, 1, 2, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 2, 1, 1, 1, 1, 1, 0, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 2, 1, 2, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 0, 2, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2, 1, 

Several commits (2) will be pushed upstream.


[1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 0, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 0, 2, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2, 1, 

[I 2023-03-31 16:01:04,477] Trial 4 finished with value: 0.9076054064958644 and parameters: {'learning_rate': 3.380516637360642e-05, 'num_train_epochs': 4, 'seed': 29, 'per_device_train_batch_size': 32}. Best is trial 2 with value: 0.912043574742788.


In [106]:
best_run

BestRun(run_id='2', objective=0.912043574742788, hyperparameters={'learning_rate': 4.403244198762251e-05, 'num_train_epochs': 2, 'seed': 34, 'per_device_train_batch_size': 8}, run_summary=None)

You can customize the objective to maximize by passing along a `compute_objective` function to the `hyperparameter_search` method, and you can customize the search space by passing a `hp_space` argument to `hyperparameter_search`. See this [forum post](https://discuss.huggingface.co/t/using-hyperparameter-search-in-trainer/785/10) for some examples.

To reproduce the best training, just set the hyperparameters in your `TrainingArgument` before creating a `Trainer`:

In [107]:
for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

trainer.train()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'pre_classifi

Epoch,Training Loss,Validation Loss,Accuracy
1,0.295200,0.298400,0.908816
2,0.232000,0.312385,0.912044


[1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 0, 1, 2, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 0, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 0, 1, 1, 2, 1, 1, 2, 2, 1, 2, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2, 1, 

TrainOutput(global_step=4958, training_loss=0.26759666452103925, metrics={'train_runtime': 444.195, 'train_samples_per_second': 89.267, 'train_steps_per_second': 11.162, 'total_flos': 575011388184876.0, 'train_loss': 0.26759666452103925, 'epoch': 2.0})

Push the model to hugging face

In [108]:
trainer.push_to_hub()

Upload file runs/Mar31_14-43-24_c2b043a0ffff/events.out.tfevents.1680278583.c2b043a0ffff.787.25: 100%|########…

To https://huggingface.co/Dc26/distilbert-base-uncased-finetuned-hate_speech
   4240c74..bcbcb15  main -> main

   4240c74..bcbcb15  main -> main

To https://huggingface.co/Dc26/distilbert-base-uncased-finetuned-hate_speech
   bcbcb15..7f35d4e  main -> main

   bcbcb15..7f35d4e  main -> main



'https://huggingface.co/Dc26/distilbert-base-uncased-finetuned-hate_speech/commit/bcbcb15b65cfadd6818a206830873b15129ff4b2'

check if the model workds

In [ ]:
# train["label"].value_counts()

In [ ]:
# test["label"].value_counts()

In [109]:
from transformers import TextClassificationPipeline
# os.environ['CUDA_VISIBLE_DEVICES'] ='0'

current_model = AutoModelForSequenceClassification.from_pretrained("Dc26/distilbert-base-uncased-finetuned-hate_speech")
pipe = TextClassificationPipeline(model=current_model, tokenizer=tokenizer, return_all_scores=False)


def get_prediction(text):
  return pipe(text)[-1]["label"][-1]

/usr/local/lib/python3.9/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [110]:
%%time
test["predicted"] = test["tweet"].apply(lambda x: get_prediction(x))


CPU times: user 5min 42s, sys: 829 ms, total: 5min 43s
Wall time: 5min 52s


In [111]:
## create a confusion matrix
test["predicted"] = test["predicted"].astype(int)
test.groupby(["predicted", "label"]).size().unstack(fill_value=0)

label,0,1,2
predicted,,,
0,74,53,12
1,200,3673,68
2,26,77,774


In [117]:
import seaborn as sns
cm = test.groupby(["predicted", "label"]).size().unstack(fill_value=0)
cm

label,0,1,2
predicted,,,
0,74,53,12
1,200,3673,68
2,26,77,774


In [113]:
test["predicted"].value_counts()
# cause tpp much 2 data predicted everything as 2

1    3941
2    877 
0    139 
Name: predicted, dtype: int64

In [121]:
cm.values

array([[  74,   53,   12],
       [ 200, 3673,   68],
       [  26,   77,  774]])

### Conclusion
- not that good in hate speech (low precision:0.24 and recall 0.53) 
  - identifying it as offensive langauge even if it's hate speech 
  - since the data is imbalanced this is also expected
- it's good at identifying offensive language (high precision:0.96 and recall 0.93)
- it's good at identifying neither language (high precision:0.90 and recall 0.88)


notes:
- Measure precision and recall and f1 score of the model
  - precision is what proportion of things returned are actually relevant (abysmal since every cat it misidentified as a dog would be counting against that)
  - recall is how many relevant things are returned out of all of the possible actually relevant things (100% recall since all of the actual dogs it returned were 100% of the actual dogs it could have returned)


In [123]:
from operator import truediv
import numpy as np

tp = np.diag(cm.values)
prec = list(map(truediv, tp, np.sum(cm.values, axis=0)))
rec = list(map(truediv, tp, np.sum(cm.values, axis=1)))
print ('Precision: {}\nRecall: {}'.format(prec, rec))

Precision: [0.24666666666666667, 0.9658164606889298, 0.9063231850117096]
Recall: [0.5323741007194245, 0.9319969550875412, 0.8825541619156214]


In [114]:
test[test["predicted"]==test["label"]].shape[0] / test.shape[0]

0.912043574742788

## Reccomendation

1. Further data cleaning
  - looking at the tweets since there are a lot of emojis so, maybe the performance could have been better if we remove emojis and done further cleaning
  - remove @username and make a generic tag instead of the model seeing different username
2.  test on the if the model would be better with the undersampling and over sample datasets
3. Compare performance with an existing paper (papers: https://github.com/aymeam/Datasets-for-Hate-Speech-Detection)